In [1]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from io import StringIO
import pandas as pd
from sqlalchemy import create_engine, text
engine = create_engine('sqlite://', echo=False)

database ='database2.sqlite'

conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables
def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """

    query_output = engine.execute(query).fetchall()
    column_names = engine.execute(query)._metadata.keys
    return pd.DataFrame(query_output, columns = column_names)

1. Country: Hierin kan je de naam vinden van verschilende landen.

2.    League: Hierin kan je informatie vinden over de verschillende Leagues/Competities, zoals de naam van de competitie en het land waarin deze wordt gehouden.

3.    Match: Hierin is de datum van een wedstrijd te vinden, samen met informatie over welk team thuis en welk team weg speelt en de eindscore van de wedstrijden

4.    Player: Hierin is te zien wat de nationaliteit, postitie, geboortedatum en naam is van de spelers

5.    Player_Attributes: Hierin is te zien over allespelers wat de sterke en slechte kanten zijn van deze spelers, hieronder valt: snelheid, balcontrole en schietvermogen.

6.    Team: hiernin zie je de namen van de clubs/teams, de landen van de clubs/teams en de gespeelde seizoenen.

7.    Team_Attributes: hierin is te zien wat de sterke en slechte kanten van de teams zijn, onder andere de sterkte van de aanval, verdedeging, creativiteit.

In [2]:
pd.read_sql("""
SELECT t.team_long_name AS team_naam, t.team_api_id AS team_id, l.name AS competitie_naam, l.country_id AS competitie_id
FROM Team AS t
JOIN Match AS m ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
JOIN League AS l ON m.league_id = l.id
WHERE t.team_long_name = 'Arsenal' """, conn)

    """
    Query the database to see in which league arsenal plays and to get information about Arsenal.

    Parameters:
    conn : A connection to the database.

    Returns:
    DataFrame: A DataFrame having these collumns:
        - team_naam : The name of Arsenal.
        - team_id : The unique number/id for Arsenal.
        - competitie_naam : The name of the league in which Arsenal plays.
        - competitie_id : The unique number/id for the league in which Arsenal plays.
    """



In [3]:
pd.read_sql("""
SELECT *
FROM Match
WHERE home_team_api_id = '9825' OR away_team_api_id = '9825'""",conn)

    """
    Get all data from the Match table where the home_team_api_id or away_team_api_id is equal to the given team_api_id.

    Parameters:
    conn : A connection  to the database.
    team_api_id : the team API ID.

    Returns:
    DataFrame: A DataFrame containing all data from the Match table where the home_team_api_id or away_team_api_id is equal to the team_api_id of arsenal.
    """
    query = f"""
            SELECT *
            FROM Match
            WHERE home_team_api_id = '{team_api_id}' OR away_team_api_id = '{team_api_id}'
            """
    return pd.read_sql(query, conn)

In [5]:
def get_arsenal_stats():
    """
   Function that Connects to a database and retrieves statistics for Arsenal
    during the 2015/2016 season.
    
    Returns a dictionary containing the following statistics:
    - total_matches: the number of matches Arsenal played
    - total_points: the number of points Arsenal earned
    - avg_points_per_match: the amount of points Arsenal averaged per match
    - home_win_percentage: the percentage of home matches Arsenal won
    - home_loss_percentage: the percentage of home matches Arsenal lost
    - home_draw_percentage: the percentage of home matches Arsenal drew
    - away_win_percentage: the percentage of away matches Arsenal won
    - away_loss_percentage: the percentage of away matches Arsenal lost
    - away_draw_percentage: the percentage of away matches Arsenal drew
    
    Returns: Win, Loss, Draw percentages And information about the point and goals in the season.
    """
conn = sqlite3.connect('database.sqlite')

cur = conn.cursor()

cur.execute("""SELECT home_team_goal, away_team_goal, home_team_api_id, away_team_api_id, season
               FROM Match
               WHERE season = '2015/2016' AND (home_team_api_id = 9825 OR away_team_api_id = 9825)""")

arsenal_home_goals = 0
arsenal_away_goals = 0
home_wins = 0
home_draws = 0
home_losses = 0
away_wins = 0
away_draws = 0
away_losses = 0

for row in cur.fetchall():
    if row[2] == 9825:
        arsenal_home_goals += row[0]
        if row[0] > row[1]:
            home_wins += 1
        elif row[0] < row[1]:
            home_losses += 1
        else:
            home_draws += 1
    else:
        arsenal_away_goals += row[1]
        if row[1] > row[0]:
            away_wins += 1
        elif row[1] < row[0]:
            away_losses += 1
        else:
            away_draws += 1
total_points = (home_wins + away_wins) * 3 + (home_draws + away_draws)

total_matches = home_wins + home_draws + home_losses + away_wins + away_draws + away_losses
avg_points_per_match = total_points / total_matches

home_win_percentage = (home_wins / (home_wins + home_draws + home_losses)) * 100
home_loss_percentage = (home_losses / (home_wins + home_draws + home_losses)) * 100
home_draw_percentage = (home_draws / (home_wins + home_draws + home_losses)) * 100

away_win_percentage = (away_wins / (away_wins + away_draws + away_losses)) * 100
away_loss_percentage = (away_losses / (away_wins + away_draws + away_losses)) * 100
away_draw_percentage = (away_draws / (away_wins + away_draws + away_losses)) * 100

print("Totaal aantal wedstrijden van Arsenal: {}".format(total_matches))
print("Totaal aantal punten van Arsenal: {}".format(total_points))
print("gemiddeld aantal punten van Arsenal : {}".format(avg_points_per_match))
print("winst percentage van Arsenal thuis: {}".format(home_win_percentage))
print("verlies percentage van Arsenal thuis: {}".format(home_loss_percentage))
print("gelijk percentage van Arsenal thuis: {}".format(home_draw_percentage))
print("winst percentage van Arsenal uit: {}".format(away_win_percentage))
print("verlies percentage van Arsenal uit: {}".format(away_loss_percentage))
print("gelijk percentage van Arsenal uit: {}".format(away_draw_percentage))

OperationalError: no such table: Match

In [ ]:

def get_league_points(database_file, league_id, season):
    conn = sqlite3.connect(database_file)
    cur = conn.cursor()

    cur.execute("""SELECT home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, season
                   FROM Match
                   WHERE season = ? AND league_id = ?""", (season, league_id)) 

    points = {}
    for row in cur.fetchall():
        home_team = row[0]
        away_team = row[1]
        home_score = row[2]
        away_score = row[3]

        if home_score > away_score:
            points[home_team] = points.get(home_team, 0) + 3
        elif home_score == away_score:
            points[home_team] = points.get(home_team, 0) + 1
            points[away_team] = points.get(away_team, 0) + 1
        else:
            points[away_team] = points.get(away_team, 0) + 3

    sorted_points = sorted(points.items(), key=lambda x: x[1], reverse=True)

    results = []
    for i, (team_id, team_points) in enumerate(sorted_points):
        cur.execute("SELECT team_long_name FROM Team WHERE team_api_id = ?", (team_id,))
        team_name = cur.fetchone()[0]
        results.append((team_name, team_points))

    conn.close()

    return results
results = get_league_points('database.sqlite', 1729, '2015/2016')
print("Ranglijst:")
for i, (team_name, team_points) in enumerate(results):
    print("{0}. {1}: {2} punten".format(i+1, team_name, team_points))
    
"""Function that retrieves the league points for each team in a given league and season from a SQLite database and returns them in a list of tuples sorted by the total number of points.

Parameters:
database_file : The path and filename of the SQLite database file.

league_id : The ID of the league to retrieve the data for.

season : The season to retrieve the data for, in the format 'yyyy/yyyy'.

Returns:
results : A list of tuples, where each tuple contains the team name and their total number of points. The list is sorted by the total number of points, in descending order.